# Init

In [20]:
import vertexai
from vertexai.generative_models import GenerativeModel, SafetySetting
import google.generativeai as genai
from pprint import pprint as pp
import json
import time
from dotenv import load_dotenv
import re
import os
import math
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup
import PyPDF2

load_dotenv()

True

In [21]:
genai.configure(api_key=os.getenv('GEMINI_API_KEY'))
gemini15pro = genai.GenerativeModel("gemini-1.5-pro")

# Prompts

## Splitted text prompt

### Format

In [22]:
split_text_format = '''{
    question: str,
    textPart: str,
    answers: [
        {
            answer: str,
            isCorrect: boolean,
        }
    ]
}'''

In [23]:
topics = '''1. questions about the personalities of the characters
2. questions about the appearance or look of characters
3. what the characters did in certain situations
4. what the characters did in certain situations, what they thought or felt
5. the attitude of some characters to other characters
6. names of cities or locations where the events took place
7. what are the key characters'''

### Prompt

In [24]:
split_text_prompt = '''Your task is to read a part of the book carefully and create only ONE question for the children who have read this part.
Question should have 4 answer choices with 1 correct answer.

Here are some topics to create a question:
1. questions about the personalities of the characters
2. questions about the appearance or look of characters
3. what the characters did in certain situations
4. what the characters did in certain situations, what they thought or felt
5. the attitude of some characters to other characters
6. names of cities or locations where the events took place
7. what are the key characters

Try to make question and answer choices more similar to the text itself.
Avoid creating tricky questions.

The question must be in this json format:
```
{}
```

The textPart field of the question should contain a part of the text from which you took the question

Here is a part of a book:
{}

The question must be in the same language as the book
'''

In [25]:
gen_prompt_template = '''You are an experienced educational content creator specializing in reading comprehension exercises for children. Your task is to create a single multiple-choice question based on a given book excerpt.

Here's the book excerpt you'll be working with:

<book_excerpt>
{}
</book_excerpt>

Now, let's review the list of topics that may be relevant to the question you'll create:

<topics_list>
{}
</topics_list>

Your task is to carefully read this excerpt and create ONE question for children who have read this part. Follow these guidelines:

1. Create a question that is closely related to the content of the text.
2. Generate 4 answer choices, with only 1 correct answer.
3. Ensure that the question and answer choices are similar in language and style to the text itself.
4. Avoid creating tricky or intentionally misleading questions or answers.
5. The question must be in the same language as the book excerpt.

You must format your output as a JSON object. Here's the required format:

<json_format>
{}
</json_format>

Important: The "textPart" field of the question should contain the specific part of the text from which you derived the question.

Before providing your final answer, wrap your thought process inside question_development tags. Consider the following steps:

1. Write down 2-3 key quotes from the excerpt that could be potential question sources.
2. List out 3-4 potential topics from the provided topic list that relate to these quotes.
3. For each potential question:
   - Formulate a clear and concise question.
   - Write down the correct answer and three plausible but incorrect answers.
   - Evaluate the question based on clarity, relevance, and difficulty level.
4. Choose the best question based on your evaluation.
5. Select the relevant text part that corresponds to your chosen question.

After your thought process, provide the final JSON output with your question, answers, and relevant text part.'''

In [26]:
gen_prompt_template_2 = '''You are an experienced educational content creator specializing in reading comprehension exercises for children. Your task is to create a single, clear, and specific multiple-choice question based on a given book excerpt.

First, carefully read the following book excerpt:

<book_excerpt>
{}
</book_excerpt>

Now, review this list of topics that may be relevant to the question you'll create:

<topics_list>
{}
</topics_list>

Your goal is to create ONE question for children who have read this excerpt. Follow these guidelines:

1. The question must be closely related to the content of the text.
2. Generate 4 answer choices, with only 1 correct answer.
3. Use language and style similar to the text itself.
4. Avoid creating tricky or intentionally misleading questions or answers.
5. The question must be in the same language as the book excerpt.
6. Ensure the question is clear and specific, explicitly mentioning any relevant context from the excerpt.

Before creating your final question, wrap your reasoning process in <question_development> tags. Follow these steps:

1. Identify 2-3 key quotes from the excerpt that could be potential question sources. Write these quotes down verbatim.
2. List 3-4 relevant topics from the provided topic list that relate to these quotes.
3. For each potential question:
   - Formulate a clear and concise question, ensuring it includes specific context.
   - Write the correct answer and three plausible but incorrect answers.
   - Evaluate the question based on clarity, relevance, and difficulty level.
   - Consider the age-appropriateness of the question for children.
   - Assess how well the question aligns with the provided topics.
4. Choose the best question based on your evaluation.
5. Select the relevant text part that corresponds to your chosen question.
6. Explain why the chosen question is the best option, considering all factors evaluated.

After your question development process, provide the final output as a JSON object. Here's the required format:

<json_format>
{}
</json_format>

Remember, the "textPart" field should contain the specific part of the text from which you derived the question.'''

# Utils

## Get formatted splitted text prompt

In [27]:
def get_split_text_prompt(text_part: str):
    return split_text_prompt.format(split_text_format, text_part)

In [28]:
def get_gen_prompt(text_part: str):
    return gen_prompt_template_2.format(text_part, topics, split_text_format)

## Remove new lines

In [29]:
def remove_extra_newlines(text):
    return re.sub(r'\n+', '\n', text)

## Read PDF

In [30]:
def read_pdf(file_path: str):
    try:
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                text += page.extract_text()
            return text
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

## Read EPUB

In [31]:
def read_epub(file_path):
    book = epub.read_epub(file_path)
    text_content = []

    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            soup = BeautifulSoup(item.get_body_content(), 'html.parser')
            text_content.append(soup.get_text())

    text = '\n'.join(text_content)
    return remove_extra_newlines(text)

## Split text

In [32]:
def split_text_into_parts(text: str, num_parts: int):
    paragraphs = re.split('\n', text.strip())

    paragraphs_per_part = math.ceil(len(paragraphs) / (num_parts))

    parts = []
    current_part = ""

    for i, paragraph in enumerate(paragraphs):
        current_part += paragraph + "\n"
        if (i + 1) % paragraphs_per_part == 0 or (i + 1) == len(paragraphs):
            parts.append(current_part.strip())
            current_part = ""

    return parts

## Split text with overlap

In [33]:
def split_text_into_parts_with_overlap(text: str, num_parts: int):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text.strip())

    sentences_per_part = math.ceil(len(sentences) / num_parts)

    parts = []
    for i in range(num_parts):
        start_index = max(0, i * sentences_per_part - 1)
        end_index = min(len(sentences), (i + 1) * sentences_per_part + 1)

        part = " ".join(sentences[start_index:end_index])
        parts.append(part.strip())

    return parts

## Gemini generate

In [34]:
def gemini_generate(model: genai.GenerativeModel, prompt: str):
    response = model.generate_content(
        prompt,
        generation_config = genai.GenerationConfig(
            temperature=0.0,
        )
    )

    return response

# Retrieve JSON from response

In [35]:
def get_json_from_response(text):
    json_match = re.search(r"<json_format>(.*?)</json_format>", text, re.DOTALL)
    if json_match:
        json_str = json_match.group(1).strip()
    else:
        markdown_match = re.search(r"```json(.*?)```", text, re.DOTALL)
        if markdown_match:
            json_str = markdown_match.group(1).strip()
        else:
            raise ValueError("JSON не найден ни в формате <json_format>, ни в формате Markdown.")

    try:
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        raise ValueError(f"Ошибка декодирования JSON: {e}")

# Generate answers

In [36]:
text = read_epub('../../poems/taras-shevchenko-katerina.epub')
parts = split_text_into_parts(text, 10)

input_tokens = 0
output_tokens = 0

questions = []

for part in parts:
    prompt = get_gen_prompt(part)
    response = gemini_generate(gemini15pro, prompt)
    input_tokens += response.usage_metadata.prompt_token_count
    output_tokens += response.usage_metadata.candidates_token_count

    json_part = get_json_from_response(response.text)
    questions.append(json_part)

    print(part)
    print("--------------------------------")
    print(response.text)
    print("================================")

with open('../../testing_data/katerina/gemini-1-5-pro-katerina.json', 'w', encoding='utf-8') as f:
    json.dump(questions, f, ensure_ascii=False, indent=4)

Василию Андреевичу Жуковскому на память 22 апреля 1838 года
І
Кохайтеся, чорнобриві,
Та не з москалями,
Бо москалі — чужі люде,
Роблять лихо з вами.
Москаль любить жартуючи,
Жартуючи кине;
Піде в свою Московщину,
А дівчина гине…
Якби сама, ще б нічого,
А то й стара мати,
Що привела на світ Божий,
Мусить погибати.
Серце в’яне співаючи,
Коли знає, за що;
Люде серця не побачать,
А скажуть — ледащо!
Кохайтеся ж, чорнобриві,
Та не з москалями,
Бо москалі — чужі люде,
Згнущаються вами.
Не слухала Катерина
Ні батька, ні неньки,
Полюбила москалика,
Як знало серденько.
Полюбила молодого,
В садочок ходила,
Поки себе, свою долю
Там занапастила.
Кличе мати вечеряти,
А донька не чує; 93
Де жартує з москаликом,
Там і заночує.
Не дві ночі карі очі
Любо цілувала,
Поки слава на все село
Недобрая стала.
Нехай собі тії люде,
Що хотять, говорять:
Вона любить, то й не чує,
Що вкралося горе.
Прийшли вісти недобрії —
В поход затрубили.
Пішов москаль в Туреччину;
Катрусю накрили.
Незчулася, та й байдуже,
Що к

In [38]:
print(input_tokens)
print(output_tokens)

12836
8051
